# **TELCO CUSTOMER SEGMENTATION**
To better understand customer performance, non-churning customers are analyzed in greater detail to identify high- and low-performing segments.

---

In [ ]:
USE ROLE ACCOUNTADMIN;

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import numpy as np
from datetime import datetime

#visualization
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer

# encoding
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from category_encoders import OrdinalEncoder, BinaryEncoder

# scaling
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler 

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# export model
import pickle
import joblib

# import warnings
# warnings.filterwarnings('ignore')

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
WITH LIST_DATES AS (
    SELECT 
        MAX(PREDICTION_DATE) AS dates
    FROM TELCO.DATAMART.TB_F_CUSTOMER_SEGMENTATION
)
SELECT 
    -- a.PREDICTION_DATE
    a.CUSTOMER_ID
    ,b.GENDER
    ,b.SENIOR_CITIZEN
    ,b.PARTNER
    ,b.DEPENDENTS
    ,b.STATE
    ,b.CITY
    ,b.LATITUDE
    ,b.LONGITUDE
    ,f.QUARTER AS QUARTER_JOINED
    ,c.PHONE_SERVICE
    ,c.MULTIPLE_LINES
    ,c.INTERNET_SERVICE
    ,c.ONLINE_SECURITY
    ,c.ONLINE_BACKUP
    ,c.DEVICE_PROTECTION
    ,c.TECH_SUPPORT
    ,c.STREAMING_TV
    ,c.STREAMING_MOVIES
    ,c.CONTRACT_TYPE
    ,c.PAPERLESS_BILLING
    ,c.PAYMENT_METHOD
    ,DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) AS TENURE_MONTHS
    ,d.MONTHLY_CHARGES
    ,DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) * d.MONTHLY_CHARGES AS TOTAL_CHARGES
    ,CASE 
        WHEN DATEDIFF('YEAR', b.DATE_JOINED, a.PREDICTION_DATE) <= 1 THEN 1 * (DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) * d.MONTHLY_CHARGES)
        ELSE DATEDIFF('YEAR', b.DATE_JOINED, a.PREDICTION_DATE) * (DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) * d.MONTHLY_CHARGES)
    END AS CLTV 
FROM TELCO.DATAMART.TB_R_CHURN_PREDICTION a
LEFT JOIN TELCO.DATAMART.TB_R_CUSTOMER b ON a.customer_id = b.customer_id
LEFT JOIN TELCO.DATAMART.TB_F_SERVICE_USAGE c ON a.customer_id = c.customer_id
LEFT JOIN TELCO.DATAMART.TB_F_REVENUE d ON a.customer_id = d.customer_id
LEFT JOIN TELCO.DATAMART.TB_R_DATE f ON d.date_joined = f.date_id
LEFT JOIN LIST_DATES g ON a.PREDICTION_DATE = g.dates
WHERE a.prediction_results = 'No'
    AND g.dates IS NULL

In [ ]:
df = non_churning_customers.to_pandas()
df_segmentation = df.copy()

df_segmentation = df_segmentation.drop(columns= 'CUSTOMER_ID')

In [ ]:
X = df_segmentation

transformer = ColumnTransformer([
    ('onehot', OneHotEncoder(drop='first'), [
        'GENDER', 'SENIOR_CITIZEN', 'PARTNER', 'DEPENDENTS', 'PHONE_SERVICE', 'MULTIPLE_LINES', 'INTERNET_SERVICE', 'ONLINE_SECURITY', 'ONLINE_BACKUP', 'DEVICE_PROTECTION', 'TECH_SUPPORT', 'STREAMING_TV', 'STREAMING_MOVIES', 'CONTRACT_TYPE', 'PAPERLESS_BILLING', 'PAYMENT_METHOD' 
    ]),
    ('binary', BinaryEncoder(), [
        'STATE', 'CITY', 'QUARTER_JOINED'
    ]),
    ('robust', RobustScaler(), [
        'LATITUDE', 'LONGITUDE', 'TENURE_MONTHS', 'MONTHLY_CHARGES', 'TOTAL_CHARGES', 'CLTV'
    ])
], remainder='passthrough')

X_scaled = transformer.fit_transform(X)

In [ ]:
# define PCA
pca = PCA()

# fit
pca.fit(X_scaled)

# transform
X_pca = pca.transform(X_scaled)

# PCA variance 
df_pca = pd.DataFrame()
df_pca['PC'] = range(1,43,1)
df_pca['variance'] = pca.explained_variance_ratio_.round(3)
df_pca['cumsum_variance'] = pca.explained_variance_ratio_.cumsum()

plt.figure(figsize= (15,5))

# membuat grid
sns.set_style('whitegrid')

sns.lineplot(data= df_pca, x= 'PC', y= 'cumsum_variance')
sns.scatterplot(data= df_pca, x= 'PC', y= 'cumsum_variance')

plt.xticks(df_pca['PC'])
;

In [ ]:
# --- 1️⃣ Fit your transformer ---
transformer.fit(X)

# --- 2️⃣ Helper: extract transformed feature names ---
def get_feature_names(column_transformer):
    output_features = []
    for name, trans, cols in column_transformer.transformers_:
        if name == 'remainder':
            continue
        if hasattr(trans, 'get_feature_names_out'):
            names = trans.get_feature_names_out(cols)
        else:
            names = cols
        output_features.extend(names)
    return output_features

X_transformed_cols = get_feature_names(transformer)

# --- 3️⃣ Apply PCA ---
pca = PCA()
X_pca = pca.fit_transform(transformer.transform(X))

# --- 4️⃣ Find top 3 features per component ---
list_imp = []
for i in range(pca.components_.shape[0]):
    pc_i_importance = pd.DataFrame({
        'feature': X_transformed_cols,
        'coef': np.abs(pca.components_[i])
    }).sort_values(by='coef', ascending=False).head(3)
    
    # Store top 3 features for that PC
    list_imp.append(list(pc_i_importance['feature'].values))

# --- 5️⃣ Build PCA summary dataframe ---
df_pca = pd.DataFrame({
    'PC': range(1, pca.n_components_ + 1),
    'variance': pca.explained_variance_ratio_.round(3),
    'cumsum_variance': pca.explained_variance_ratio_.cumsum()
})

# Ensure feature importance columns are cleanly added
df_pca = df_pca.drop(columns=['feature_importance', 'Top1', 'Top2', 'Top3'], errors='ignore')

top_features = pd.DataFrame(list_imp, columns=['Top1', 'Top2', 'Top3'])
df_pca = pd.concat([df_pca, top_features], axis=1)

# --- 6️⃣ Display result ---
df_pca.head(15)

In [ ]:
pca_11com = PCA(n_components= 11)

# fit and transform
X_pca_11com = pca_11com.fit_transform(X_scaled)

df_11com = pd.DataFrame(data= X_pca_11com, columns= ['PC0', 'PC1','PC2', 'PC3','PC4', 'PC5','PC6', 'PC7','PC8', 'PC9', 'PC10'])
df_11com.head()

In [ ]:
list_n_cluster = range(2, 10)
list_inertia = []

for i in list_n_cluster:
    # define model
    kmeans = KMeans(n_clusters=i, random_state=0)

    # fit on PCA data
    kmeans.fit(df_11com)

    # inertia (within-cluster sum of squares)
    inertia = kmeans.inertia_
    list_inertia.append(inertia)

# visualize elbow and silhouette
plt.figure(figsize=(10,5))
plt.plot(list_n_cluster, list_inertia, 'o-', label='Inertia (Elbow)')
plt.tight_layout()
plt.legend()
plt.show()

In [ ]:
kmeans_best = KMeans(n_clusters= 4, random_state= 0)

kmeans_best.fit(X_scaled)

df_clustered = pd.DataFrame()
df_clustered['CUSTOMER_ID'] = df['CUSTOMER_ID']
df_clustered['CUSTOMER_CLUSTER'] = kmeans_best.labels_

df_clustered.head(10)

In [ ]:
CREATE OR REPLACE PROCEDURE TELCO.DATAMART.SP_CUSTOMER_SEGMENTATION()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.9'
PACKAGES = ('snowflake-snowpark-python', 'pandas', 'scikit-learn', 'category_encoders')
HANDLER = 'main'
AS
$$
import pandas as pd
from snowflake.snowpark.functions import col
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from category_encoders.binary import BinaryEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

def main(session) -> str:

    # ================
    # 1️⃣ Load Data
    # ================
    query = """
        WITH LIST_DATES AS (
            SELECT 
                MAX(PREDICTION_DATE) AS dates
            FROM TELCO.DATAMART.TB_F_CUSTOMER_CLUSTER
        )
        SELECT 
            a.CUSTOMER_ID,
            b.GENDER,
            b.SENIOR_CITIZEN,
            b.PARTNER,
            b.DEPENDENTS,
            b.STATE,
            b.CITY,
            b.LATITUDE,
            b.LONGITUDE,
            f.QUARTER AS QUARTER_JOINED,
            c.PHONE_SERVICE,
            c.MULTIPLE_LINES,
            c.INTERNET_SERVICE,
            c.ONLINE_SECURITY,
            c.ONLINE_BACKUP,
            c.DEVICE_PROTECTION,
            c.TECH_SUPPORT,
            c.STREAMING_TV,
            c.STREAMING_MOVIES,
            c.CONTRACT_TYPE,
            c.PAPERLESS_BILLING,
            c.PAYMENT_METHOD,
            DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) AS TENURE_MONTHS,
            d.MONTHLY_CHARGES,
            DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) * d.MONTHLY_CHARGES AS TOTAL_CHARGES,
            CASE 
                WHEN DATEDIFF('YEAR', b.DATE_JOINED, a.PREDICTION_DATE) <= 1 THEN 
                    1 * (DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) * d.MONTHLY_CHARGES)
                ELSE 
                    DATEDIFF('YEAR', b.DATE_JOINED, a.PREDICTION_DATE) * 
                    (DATEDIFF('MONTH', b.DATE_JOINED, a.PREDICTION_DATE) * d.MONTHLY_CHARGES)
            END AS CLTV 
        FROM TELCO.DATAMART.TB_R_CHURN_PREDICTION a
        LEFT JOIN TELCO.DATAMART.TB_R_CUSTOMER b ON a.customer_id = b.customer_id
        LEFT JOIN TELCO.DATAMART.TB_F_SERVICE_USAGE c ON a.customer_id = c.customer_id
        LEFT JOIN TELCO.DATAMART.TB_F_REVENUE d ON a.customer_id = d.customer_id
        LEFT JOIN TELCO.DATAMART.TB_R_DATE f ON d.date_joined = f.date_id
        LEFT JOIN LIST_DATES g ON a.PREDICTION_DATE = g.dates
        WHERE a.prediction_results = 'No'
          AND g.dates IS NULL
    """
    df = session.sql(query).to_pandas()

    # ================
    # 2️⃣ Preprocess
    # ================
    df_seg = df.copy()
    customer_ids = df_seg['CUSTOMER_ID']
    df_seg = df_seg.drop(columns='CUSTOMER_ID')

    transformer = ColumnTransformer([
        ('onehot', OneHotEncoder(drop='first', handle_unknown='ignore'), [
            'GENDER', 'SENIOR_CITIZEN', 'PARTNER', 'DEPENDENTS',
            'PHONE_SERVICE', 'MULTIPLE_LINES', 'INTERNET_SERVICE',
            'ONLINE_SECURITY', 'ONLINE_BACKUP', 'DEVICE_PROTECTION',
            'TECH_SUPPORT', 'STREAMING_TV', 'STREAMING_MOVIES',
            'CONTRACT_TYPE', 'PAPERLESS_BILLING', 'PAYMENT_METHOD'
        ]),
        ('binary', BinaryEncoder(), ['STATE', 'CITY', 'QUARTER_JOINED']),
        ('robust', RobustScaler(), [
            'LATITUDE', 'LONGITUDE', 'TENURE_MONTHS',
            'MONTHLY_CHARGES', 'TOTAL_CHARGES', 'CLTV'
        ])
    ], remainder='passthrough')

    X_scaled = transformer.fit_transform(df_seg)

    # ================
    # 3️⃣ PCA (11 Components)
    # ================
    pca = PCA(n_components=11)
    X_pca = pca.fit_transform(X_scaled)

    # ================
    # 4️⃣ K-Means (4 Clusters)
    # ================
    kmeans = KMeans(n_clusters=4, random_state=0)
    clusters = kmeans.fit_predict(X_scaled)

    df_result = pd.DataFrame({
        'CUSTOMER_ID': customer_ids,
        'CUSTOMER_CLUSTER': clusters
    })

    # Add prediction date from Snowflake
    df_result["PREDICTION_DATE"] = session.sql("SELECT CURRENT_DATE()").collect()[0][0]
    
    # (Optional) Add a readable status or label mapping
    df_result["PREDICTED_STATUS"] = df_result["CUSTOMER_CLUSTER"].replace({
        0: "Segment 1",
        1: "Segment 2",
        2: "Segment 3",
        3: "Segment 4"
    })
    df_export = pd.DataFrame()
    df_export['PREDICTION_DATE'] = df_result['PREDICTION_DATE']
    df_export['CUSTOMER_ID'] = df_result['CUSTOMER_ID']
    df_export['CUSTOMER_SEGMENTATION'] = df_result['PREDICTED_STATUS']

    # ================
    # 5️⃣ Save Result Back to Snowflake
    # ================
    session.create_dataframe(df_export).write.mode("append").save_as_table("TELCO.DATAMART.TB_F_CUSTOMER_CLUSTER")

    return f"✅ Customer segmentation completed successfully! Total customers clustered: {len(df_export)}"
$$;